In [57]:
import sys

In [58]:
from importlib import reload

In [2]:
sys.path.append(os.path.join(os.path.abspath(os.path.join('../..')), 'src'))

In [43]:
import mysql_utils
import events_merge
import doc_proc

In [59]:
import spacy
from spacy import attrs
import textacy

In [62]:
reload(mysql_utils)

<module 'mysql_utils' from '/home/immersinn/gits/rssfeed_link_collector/src/mysql_utils.py'>

In [15]:
cnx = mysql_utils.getCnx()
cur = mysql_utils.getCur(cnx)

In [14]:
limit = 15

In [16]:
query_text = '''SELECT * FROM {} LIMIT {}'''.format(mysql_utils.TABLE, limit)

In [41]:
class DocProcessor():
    
    def __init__(self,):
        preproc = lambda text: textacy.preprocess.preprocess_text(text,
                                                                  no_contractions=True,
                                                                  no_numbers=True,
                                                                  no_emails=True
                                                                  )
        nlp = spacy.load("en", add_vectors=False)
        nlp.pipeline = [nlp.tagger, nlp.parser]
        
        self.preproc = preproc
        self._nlp = nlp
        self.process = self._nlp.make_doc
    
    
    def doc2BOW(self, doc):
        return(self._bow_transform(doc))
    
    
    def _bow_transform(self, doc):
        doc = self.preproc(doc)
        doc = self.process(doc)
        bow = {self._nlp.vocab[k].lower_ : v \
               for k,v in doc.count_by(attrs.LOWER).items()}
        return(bow)

In [44]:
dd = lambda doc: doc_proc.build_text_feature(doc, 
                                             components = ['title', 'summary'],
                                             lower=False, 
                                             remove_stops=False,
                                             html_text=True)

In [39]:
dp = DocProcessor()

In [70]:
cur.execute(query_text)
bows = []

while True:

    try:
        doc_dict = mysql_utils.dictDocFromCursor(cur)
        doc = dd(doc_dict)
        bow = dp.doc2BOW(doc)
        bows.append(bow)

    except StopIteration:
        break

    except KeyboardInterrupt:
        raise KeyboardInterrupt

In [71]:
bows[5]

{'draft': 1,
 'female': 1,
 'how': 1,
 'matters': 1,
 'nfl': 1,
 'olympian': 1,
 'prospects': 1,
 'shows': 1,
 'speed': 1,
 'that': 1}

In [72]:
cnx.close()

In [103]:
doc_id = doc_dict['id']
bow = bows[-1]

## Insert Words, Counts into respective Tables

In [163]:
class TDMTHandle():
    
    def __init__(self,):
        self.cnx = mysql_utils.getCnx()
        self.cur = mysql_utils.getCur(self.cnx)
        
    def insertDocBOW(self, bow, doc_id):
    
        for word in bow:
            try:
                # Step 01:
                    # insert word if new
                    # get word key
                word_id = self._word_get_key(word)
                
                # Step 02:
                    # insert counts
                self._insert_count(doc_id, word_id, bow[word])
            except mysql_utils.IntegrityError:
                self.cnx.rollback()
            
    def _word_get_key(self, word):
        word_query = """SELECT id FROM words WHERE word = '{}' LIMIT 1""".format(word)
        self.cur.execute(word_query)
        key = self.cur.fetchone()
        
        if not key:
            word_insert = """INSERT INTO words (word) VALUES ('{}')""".format(word)
            self.cur.execute(word_insert)
            self.cnx.commit()
            self.cur.execute(word_query)
            key = self.cur.fetchone()
            
        key = key[0]
            
        return(key)
    
    def _insert_count(self, doc_id, word_id, count):
        count_insert = ("INSERT INTO doc_bows "
                        "(word_id, doc_id, wcount) "
                        "VALUES ({0}, {1}, {2})".format(word_id, doc_id, count)
                       )
        self.cur.execute(count_insert)
        self.cnx.commit()

In [173]:
th = TDMTHandle()

In [174]:
th.insertDocBOW(doc_id=doc_id, bow=bow)

IntegrityError: 1062 (23000): Duplicate entry '1-15' for key 'PRIMARY'

In [171]:
th.cnx.close()

In [3]:
27 % 9

0

In [4]:
32000 / 100 * 4 / 60

21.333333333333332